In [27]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler

In [28]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

scaler = StandardScaler()

train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

train_images = scaler.fit_transform(train_images)
test_images = scaler.transform(test_images)

# train_images = train_images.reshape(train_images.shape[0], 28, 28)
# test_images = test_images.reshape(test_images.shape[0], 28, 28)

train_labels = keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = keras.utils.to_categorical(test_labels, num_classes=10)

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [29]:
model = keras.models.Sequential([
    keras.layers.InputLayer(shape=(784,)),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"), # layer 1
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"), # layer 2
    keras.layers.Dense(50, activation="elu", kernel_initializer="he_normal"), # layer 3
    keras.layers.Dense(10, activation="softmax") # layer 4
])

def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)


n_epochs = 5
batch_size = 32
n_steps = len(train_images) // batch_size
optimizer_upper = keras.optimizers.Adam(learning_rate=0.001)
optimizer_lower = keras.optimizers.SGD(learning_rate=0.01)
loss_fn = keras.losses.categorical_crossentropy
mean_loss = keras.metrics.Mean()
mean_loss.name = "loss"
mean_metrics = keras.metrics.Mean()
metrics = [keras.metrics.CategoricalAccuracy()]

for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(train_images, train_labels)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        # input layer does not count as a layer with trainable variables, so it is not considered
        # when counting the layers for the purpose of applying optimizers to specific layers.
        optimizer_lower.apply_gradients(zip(gradients[:-2], model.trainable_variables[:-2])) # layers 1 and 2
        optimizer_upper.apply_gradients(zip(gradients[-2:], model.trainable_variables[-2:])) # layers 3 and 4
        mean_loss(loss)
        
        for metric in metrics:
            metric.update_state(y_batch, y_pred)
        
        if (step == n_steps):
            y_val_pred = model(val_images)
            val_loss = tf.reduce_mean(loss_fn(val_labels, y_val_pred))
            val_acc = keras.metrics.categorical_accuracy(val_labels, y_val_pred)
            val_acc_mean = tf.reduce_mean(val_acc)  # Compute mean accuracy
            print_status_bar(n_steps, n_steps, mean_loss, metrics)
            print("Validation loss: {:.4f} - Validation accuracy: {:.4f}".format(val_loss.numpy(), val_acc_mean.numpy()))
        else:
            print_status_bar(step, n_steps, mean_loss, metrics)
    
    

    for metric in [mean_loss] + metrics:
        metric.reset_state()


c:\programming\ml\my_env\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5
1500/1500 - loss: 1.7506 - categorical_accuracy: 0.4437
Validation loss: 1.1560 - Validation accuracy: 0.5746
Epoch 2/5
1500/1500 - loss: 1.0093 - categorical_accuracy: 0.6303
Validation loss: 0.9434 - Validation accuracy: 0.6477
Epoch 3/5
1500/1500 - loss: 0.8262 - categorical_accuracy: 0.6934
Validation loss: 0.7547 - Validation accuracy: 0.7306
Epoch 4/5
1500/1500 - loss: 0.7461 - categorical_accuracy: 0.7210
Validation loss: 0.6843 - Validation accuracy: 0.7491
Epoch 5/5
1500/1500 - loss: 0.7114 - categorical_accuracy: 0.7333
Validation loss: 0.8469 - Validation accuracy: 0.6718
